In [1]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from astroquery.mpc import MPC

# Cobs API

In [2]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [3]:
# Conexión con la API de COBS
nombre_cometa =  'C/2023 A3'
fecha_inicial = '1970-01-01 00:00'
Link_cops_API = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date={fecha_inicial}&exclude_faint=False&exclude_not_accurate=False'

try: 
    if verificar_conexion():
        response = requests.get(Link_cops_API)

        if response.status_code == 200:
            content = response.json()
            print('✅ Base de datos actualizada.')
            
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Base de datos actualizada.


In [4]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content['objects'])
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       2424 non-null   object 
 1   obs_date                   2424 non-null   object 
 2   comet                      2424 non-null   object 
 3   observer                   2424 non-null   object 
 4   location                   1251 non-null   object 
 5   extinction                 331 non-null    object 
 6   obs_method                 2423 non-null   object 
 7   comet_visibility           21 non-null     object 
 8   magnitude                  2422 non-null   object 
 9   conditions                 114 non-null    object 
 10  ref_catalog                2423 non-null   object 
 11  instrument_aperture        2424 non-null   object 
 12  instrument_type            2424 non-null   object 
 13  instrument_focal_ratio     1541 non-null   float

In [5]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2424
Variables: 47


In [6]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
385,V,2024-10-30 09:35:59,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Yoshimi', 'last_name': 'Nagai'...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,5.5,None,...,None,None,None,None,None,,None,NaN,,2024-10-30 13:25:01
1609,V,2024-05-26 04:33:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Carl', 'last_name': 'Hergenrot...","Tanque Verde, Arizona",None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,10.2,None,...,None,None,None,None,None,,19.6,2.0,,2024-05-26 05:32:57
1119,V,2024-10-01 08:09:36,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Marco', 'last_name': 'Goiato',...",None,None,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,2.1,None,...,None,None,None,None,None,,None,NaN,,2024-10-01 09:09:05
973,C,2024-10-10 18:06:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Mieczyslaw Leszek', 'last_name...",SOHO,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,-3.8,None,...,0.10,2.74,56.0,56.0,s,,None,NaN,HIP 61941,2024-10-11 14:44:31
1320,C,2024-07-18 17:45:35,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Thomas', 'last_name': 'Lehmann...",Hakos (remote),None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,9.3,None,...,None,None,1.5,1.5,s,"CMOS/G, mlim=18.9, moon 91% dist 95 deg",None,NaN,,2024-08-02 11:48:08


In [7]:
# Métodos de observación
cometa_df.obs_method.apply(
    lambda registro: f"{registro['key']}: {registro['name']}" if (registro is not None) and ('key' in registro) and ('name' in registro) else 'Datos faltantes'
).value_counts()

obs_method
C: Unfiltered total CCD magnitude                                                   581
M: Modified-Out method                                                              516
S: In-Out method                                                                    424
Z: CCD Visual equivalent                                                            347
B: Simple Out-Out method                                                            157
V: Johnson/Bessel/Kron/Cousins V with CCD                                           108
I: In-focus                                                                         104
k: Kron/Cousins R with CCD                                                           46
D: Johnson/Bessel/Kron/Cousins B with CCD                                            27
H: Kron/Cousins I with CCD                                                           21
-: Unknown                                                                           18
P: photographic      

In [8]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'] if registro is not None and 'key' in registro else 'Dato faltante')
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [9]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2424 non-null   object        
 1   obs_date        2424 non-null   datetime64[ns]
 2   magnitude       2422 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 56.9+ KB


In [10]:
# Con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2424
Variables: 3


In [11]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
963,S,2024-10-11,1.5
433,M,2024-10-27,5.7
1771,C,2024-05-09,11.2
778,M,2024-10-17,2.6
2121,Z,2024-03-31,11.2


In [12]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Perihelio Cobs API

In [13]:
# Conexión con la API de COBS para obtener el perihelio
try: 
    Link_cops_API = f'https://cobs.si/api/comet.api?des={nombre_cometa}'

    if verificar_conexion():
        response = requests.get(Link_cops_API)

        if response.status_code == 200:
            perihelio = pd.to_datetime(response.json()['object']['perihelion_date'])
            print('✅ Perihelio del cometa obtenido.')
    
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Perihelio del cometa obtenido.


# MPC API usando astroquery.

In [14]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days + 1 if (fecha_final - fecha_inicial).days <= 1441 else 1441

ephemeris = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = fechas)  # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2023-02-25,230.565417,-1.256111,6.946,7.290,106.6,7.5,17.6,8.17,318.9
1,2023-02-26,230.528333,-1.214722,6.921,7.281,107.6,7.4,17.6,8.51,317.4
2,2023-02-27,230.488750,-1.172778,6.896,7.272,108.6,7.4,17.6,8.86,316.0
3,2023-02-28,230.446250,-1.129722,6.872,7.264,109.6,7.4,17.6,9.21,314.7
4,2023-03-01,230.401250,-1.086389,6.847,7.255,110.6,7.3,17.6,9.56,313.4
...,...,...,...,...,...,...,...,...,...,...
653,2024-12-09,289.449583,4.830556,2.184,1.608,42.8,24.6,9.8,44.79,80.6
654,2024-12-10,289.742917,4.880278,2.211,1.624,42.3,24.1,9.9,44.22,80.2
655,2024-12-11,290.032500,4.931389,2.238,1.641,41.8,23.6,10.0,43.67,79.8
656,2024-12-12,290.318333,4.983611,2.264,1.658,41.3,23.1,10.0,43.16,79.3


In [15]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658 entries, 0 to 657
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           658 non-null    datetime64[ns]
 1   ra             658 non-null    float64       
 2   dec            658 non-null    float64       
 3   delta          658 non-null    float64       
 4   r              658 non-null    float64       
 5   elongation     658 non-null    float64       
 6   phase          658 non-null    float64       
 7   v              658 non-null    float64       
 8   proper_motion  658 non-null    float64       
 9   direction      658 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 51.5 KB


In [16]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [17]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2023-02-25,6.946,7.290,7.5
1,2023-02-26,6.921,7.281,7.4
2,2023-02-27,6.896,7.272,7.4
3,2023-02-28,6.872,7.264,7.4
4,2023-03-01,6.847,7.255,7.3
...,...,...,...,...
653,2024-12-09,2.184,1.608,24.6
654,2024-12-10,2.211,1.624,24.1
655,2024-12-11,2.238,1.641,23.6
656,2024-12-12,2.264,1.658,23.1


# Unión de las bases de datos.

In [18]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,B,2024-12-13,10.3,2.290,1.674,22.6
1,Z,2024-12-12,9.2,2.264,1.658,23.1
2,S,2024-12-12,9.4,2.264,1.658,23.1
3,S,2024-12-10,9.5,2.211,1.624,24.1
4,Z,2024-12-10,9.1,2.211,1.624,24.1
...,...,...,...,...,...,...
2419,C,2023-02-27,18.0,6.896,7.272,7.4
2420,C,2023-02-26,18.1,6.921,7.281,7.4
2421,C,2023-02-25,18.2,6.946,7.290,7.5
2422,C,2023-02-25,17.9,6.946,7.290,7.5


In [19]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2424 non-null   object        
 1   obs_date        2424 non-null   datetime64[ns]
 2   magnitude       2422 non-null   float64       
 3   delta           2424 non-null   float64       
 4   r               2424 non-null   float64       
 5   phase           2424 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 113.8+ KB


In [ ]:
# Reducción de la magnitud aparente y calculo del Delta t
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )

curva_de_luz_procesada_df['delta_t'] = (curva_de_luz_procesada_df.obs_date - perihelio) # type: ignore
curva_de_luz_procesada_df['delta_t'] = curva_de_luz_procesada_df.delta_t.apply(lambda delta_t: delta_t.days)

curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,B,2024-12-13,10.3,2.290,1.674,22.6,7.382045,76
1,Z,2024-12-12,9.2,2.264,1.658,23.1,6.327695,75
2,S,2024-12-12,9.4,2.264,1.658,23.1,6.527695,75
3,S,2024-12-10,9.5,2.211,1.624,24.1,6.724126,73
4,Z,2024-12-10,9.1,2.211,1.624,24.1,6.324126,73
...,...,...,...,...,...,...,...,...
2419,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,-579
2420,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,-580
2421,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,-581
2422,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,-581


In [21]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [22]:
# Curva de luz reducida
labels = {'delta_t':'t-Δt','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='delta_t', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [23]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,B,2024-12-13,10.3,2.290,1.674,22.6,7.382045,76,NaN
1,Z,2024-12-12,9.2,2.264,1.658,23.1,6.327695,75,NaN
2,S,2024-12-12,9.4,2.264,1.658,23.1,6.527695,75,NaN
3,S,2024-12-10,9.5,2.211,1.624,24.1,6.724126,73,NaN
4,Z,2024-12-10,9.1,2.211,1.624,24.1,6.324126,73,NaN
...,...,...,...,...,...,...,...,...,...
2419,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,-579,9.500592
2420,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,-580,9.507435
2421,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,-581,9.500014
2422,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,-581,9.494068


In [24]:
# Curva de luz Promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [25]:
# Curva de luz Promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='delta_t', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v1 (Promedio corrido -> agrupación)

In [26]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_v1_df = curva_de_luz_interna_v1_df.reset_index()

curva_de_luz_interna_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,-581,9.503923
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,9.507435
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,-579,9.500592
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,-578,9.585723
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,9.552207
...,...,...,...,...,...,...,...,...,...
386,2024-12-08,S,9.7,2.157,1.591,25.2,7.022398,71,6.722866
387,2024-12-09,Z,9.9,2.184,1.608,24.6,7.172307,72,6.631519
388,2024-12-10,Z,9.5,2.211,1.624,24.1,6.724126,73,6.661991
389,2024-12-12,Z,9.4,2.264,1.658,23.1,6.527695,75,NaN


In [27]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [28]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v1 (Promedio corrido -> agrupación)

In [29]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_v1_df = curva_de_luz_externa_v1_df.reset_index()
curva_de_luz_externa_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581,9.494068
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,9.507435
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579,9.495630
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578,9.504953
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,9.552207
...,...,...,...,...,...,...,...,...,...
386,2024-12-08,C,9.1,2.157,1.591,25.2,6.422398,71,6.623359
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72,6.623606
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73,6.510860
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75,NaN


In [30]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [31]:
# Gráfica de lus promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v2 (Agrupación  -> promedio corrido)

In [32]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_max_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_agrupada_max_v2_df = curva_de_luz_agrupada_max_v2_df.reset_index()
curva_de_luz_agrupada_max_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,-581
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,-579
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,-578
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577
...,...,...,...,...,...,...,...,...
386,2024-12-08,S,9.7,2.157,1.591,25.2,7.022398,71
387,2024-12-09,Z,9.9,2.184,1.608,24.6,7.172307,72
388,2024-12-10,Z,9.5,2.211,1.624,24.1,6.724126,73
389,2024-12-12,Z,9.4,2.264,1.658,23.1,6.527695,75


In [33]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_max_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [34]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_interna_v2_df = curva_de_luz_agrupada_max_v2_df.copy()
curva_de_luz_interna_v2_df['promedio_movil'] = curva_de_luz_interna_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_interna_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,-581,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,NaN
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,-579,NaN
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,-578,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,NaN
...,...,...,...,...,...,...,...,...,...
386,2024-12-08,S,9.7,2.157,1.591,25.2,7.022398,71,6.794863
387,2024-12-09,Z,9.9,2.184,1.608,24.6,7.172307,72,6.870259
388,2024-12-10,Z,9.5,2.211,1.624,24.1,6.724126,73,6.918422
389,2024-12-12,Z,9.4,2.264,1.658,23.1,6.527695,75,6.917863


In [35]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [36]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v2 (Agrupación  -> promedio corrido)

In [37]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_min_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').min()
curva_de_luz_agrupada_min_v2_df = curva_de_luz_agrupada_min_v2_df.reset_index()
curva_de_luz_agrupada_min_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577
...,...,...,...,...,...,...,...,...
386,2024-12-08,C,9.1,2.157,1.591,25.2,6.422398,71
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75


In [38]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_min_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [39]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_externa_v2_df = curva_de_luz_agrupada_min_v2_df.copy()
curva_de_luz_externa_v2_df['promedio_movil'] = curva_de_luz_externa_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_externa_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,NaN
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579,NaN
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,NaN
...,...,...,...,...,...,...,...,...,...
386,2024-12-08,C,9.1,2.157,1.591,25.2,6.422398,71,6.323434
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72,6.413116
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73,6.461279
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75,6.432149


In [40]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [41]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v1 (Mediana de cada día registrado).

In [42]:
# Creación del data frame curva de luz mediana v1
curva_de_luz_mediada_v1_df = curva_de_luz_procesada_df.groupby(by= 'obs_date').median(numeric_only= True)
curva_de_luz_mediada_v1_df = curva_de_luz_mediada_v1_df.reset_index()
curva_de_luz_mediada_v1_df

,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2023-02-25,18.20,6.946,7.290,7.5,9.677688,-581.0
1,2023-02-26,18.10,6.921,7.281,7.4,9.588201,-580.0
2,2023-02-27,17.95,6.896,7.272,7.4,9.448744,-579.0
3,2023-02-28,17.95,6.872,7.264,7.4,9.458705,-578.0
4,2023-03-01,17.90,6.847,7.255,7.3,9.419311,-577.0
...,...,...,...,...,...,...,...
386,2024-12-08,9.30,2.157,1.591,25.2,6.622398,71.0
387,2024-12-09,9.55,2.184,1.608,24.6,6.822307,72.0
388,2024-12-10,9.40,2.211,1.624,24.1,6.624126,73.0
389,2024-12-12,9.30,2.264,1.658,23.1,6.427695,75.0


In [43]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [44]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='delta_t', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v2 (Mediana de las dos curvas).

In [45]:
# Creación del data frame curva de luz mediana v2
curva_de_luz_mediada_v2_df = curva_de_luz_externa_v2_df.copy()
curva_de_luz_mediada_v2_df['mediana'] = (curva_de_luz_interna_v2_df['promedio_movil'] + curva_de_luz_externa_v2_df['promedio_movil'])/2
curva_de_luz_mediada_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil,mediana
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,-581,NaN,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,-580,NaN,NaN
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,-579,NaN,NaN
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,-578,NaN,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,-577,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
386,2024-12-08,C,9.1,2.157,1.591,25.2,6.422398,71,6.323434,6.559149
387,2024-12-09,S,9.2,2.184,1.608,24.6,6.472307,72,6.413116,6.641687
388,2024-12-10,B,9.1,2.211,1.624,24.1,6.324126,73,6.461279,6.689851
389,2024-12-12,S,9.2,2.264,1.658,23.1,6.327695,75,6.432149,6.675006


In [46]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='obs_date', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [47]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='delta_t', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz v1 (Promedio corrido -> agrupación)

In [58]:
# Gráfica de luz promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.obs_date, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente_v2', marker=dict(color='green', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo_v2', marker=dict(color='blue', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana_v2', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

In [59]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.delta_t, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.delta_t, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.delta_t, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

# Comparación de las curvas de luz v2 (Agrupación -> promedio corrido)

In [60]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

In [61]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.delta_t, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()